In [53]:
user_log.select(["userId","page"]).dropDuplicates().where(user_log.userId == "").show()

+------+-----+
|userId| page|
+------+-----+
|      | Home|
|      |About|
|      |Login|
|      | Help|
+------+-----+



In [61]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- hour: string (nullable = true)



there is no "NextSong" that emptry string user visit

In [55]:
user_log.select(["userId","page"]).where(user_log.userId == "").describe().show()

+-------+------+-----+
|summary|userId| page|
+-------+------+-----+
|  count|   336|  336|
|   mean|  null| null|
| stddev|  null| null|
|    min|      |About|
|    max|      |Login|
+-------+------+-----+



Max is Login

Q: How many female users do we have in the dataset?

In [60]:
user_log.select(["userId"]).dropDuplicates().where(user_log.gender == "F").count()

462

Q: How many song were played from the most played artist?

In [70]:
user_log.select('Artist') \
    .groupBy('Artist') \
    .agg({'Artist':'count'}) \
    .withColumnRenamed('count(Artist)', 'Artistcount') \
    .sort(desc('Artistcount')) \
    .show(1)

+--------+-----------+
|  Artist|Artistcount|
+--------+-----------+
|Coldplay|         83|
+--------+-----------+
only showing top 1 row



Q: How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [81]:
# TODO: filter out 0 sum and max sum to get more exact answer

function = udf(lambda ishome : int(ishome == 'Home'), IntegerType())
from pyspark.sql.window import Window
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
user_window = Window \
    .partitionBy('userID') \
    .orderBy(desc('ts')) \
    .rangeBetween(Window.unboundedPreceding, 0)

cusum = user_log.filter((user_log.page == 'NextSong') | (user_log.page == 'Home')) \
    .select('userID', 'page', 'ts') \
    .withColumn('homevisit', function(col('page'))) \
    .withColumn('period', Fsum('homevisit').over(user_window))

cusum.filter((cusum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'}).show()

+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+

